train 데이터를 테스트용과 학습용으로 구분

In [1]:
# google colab에서 사용할 데이터를 드라이브에 업로드 후 사용하기 위해 필요한 패키지
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import os, sys
# sys.path.append('/content/drive/melon_data/')
my_path = '/content/notebook'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env/', my_path)
sys.path.insert(0, my_path)
# sys.path.update()

path = '/content/drive/My Drive/melon_data/'

Mounted at /content/drive/


In [2]:
from split_data import ArenaSplitter

spliter = ArenaSplitter()
spliter.run(fname='/content/drive/My Drive/melon_data/train.json')


Reading data...

Total playlists: 115071
Splitting data...
Original train...
Original val...
Masked val...
Total: 23015, Song only: 6904, Song & Tags: 11508, Tags only: 3452, Title only: 1151


장르별 최빈도 곡과 플레이리스트의 최빈도 태그를 구하는 모듈을 통해 results.json을 생성한다.

In [3]:
from genre_most_popular import GenreMostPopular
genreMostPopular = GenreMostPopular()
genreMostPopular.run(song_meta_fname=path+'song_meta.json', train_fname='arena_data/orig/train.json', question_fname='arena_data/questions/val.json')

Loading song meta...
Loading train file...
Loading question file...
Writing answers...


100%|██████████| 23015/23015 [00:02<00:00, 10387.38it/s]


테스트용 데이터셋을 각 플레이리스트의 태그와 곡을 50%만 추출하여 questions/val.json 로 모델을 학습시킨다.


- word2vec을 학습시키기 위한 데이터 전처리 작업 수행

In [4]:
# 필요한 패키지 import
from collections import Counter  # 빈도를 셀 때 유용한 패키지

import fire

# 머신 러닝 관련 진행 상황을 확인할 수 있는 패키지
from tqdm import tqdm 

# 아레나에서 제공하는 모듈
from arena_util import load_json
from arena_util import write_json
from arena_util import remove_seen
from arena_util import most_popular


import pandas as pd
import pickle
import time

from tqdm import tqdm
# from word2vec import Word2VecTrainer
# from myimplicit import calculate_similar_playlists

In [5]:
# 데이터 로딩
song_meta_json = load_json(path + 'song_meta.json')
train_data = load_json('arena_data/orig/train.json')
questions = load_json('arena_data/questions/val.json')

In [6]:
new_train_fname = './res/train_val.json'
train = train_data
val = questions
write_json(train+val, './../res/train_val.json')

- _train(new_train_fname, test_fname)

In [7]:
# 데이터를 로딩하여 데이터 프레임화
test_fname = 'arena_data/orig/train.json'
train_fname = new_train_fname
train = pd.read_json(train_fname, encoding='UTF-8')
test = pd.read_json(test_fname, encoding='UTF-8')
data = pd.concat([train, test])

- word2vec 모델 생성       
  -  myw2v = Word2VecTrainer
(train_fname=train_fname,             test_fname=test_fname,                  most_results_fname="./arena_data/results/results.json")

In [8]:
# word2vec 모델 생성 및 학습과 관련된 패키지
import pandas as pd
import random
import pickle
import time
from tqdm import tqdm
from arena_util import remove_seen, write_json, load_json

from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, remove_stopwords, stem_text, strip_multiple_whitespaces

In [9]:
# 모델 생성을 위해 필요한 파라미터들 초깃값
train_fname="arena_data/orig/train.json"
test_fname="arena_data/questions/val.json"
most_results_fname="./arena_data/results/results.json"

train = pd.read_json(train_fname, encoding='UTF-8')
test = pd.read_json(test_fname, encoding='UTF-8')
song_meta = pd.read_json(path + "song_meta.json", encoding='UTF-8')
most_results = pd.read_json(most_results_fname, encoding='UTF-8')


- word2vec 모델 전처리 및 학습
  - myw2v.run(topn=80, song_weight=1, tag_weight=2, title_weight=4,tag_filename = 'model_tag_w1.pkl', save_model=True)

    min_count = 1 # 등장 횟수에 대한 파라미터로 빈도수가 해당 값 이하면 무시

    size = 200 # n차원짜리 벡터스페이스. 벡터 차원이 높을수록 항상 좋은 것은 아니지만 대체적으로 더 나은 모델이 될 수 있다.

    windows = 100 # 창 크기. 학습 알고리즘이 고려해야 하는 컨텍스트 단어 수

    sg=1 (1= skip-gram으로 느리지만 더 나은 결과를 낸다)

In [10]:
sg = 1 # skip-gram 사용 / 0이면 CBOW 사용
topn=150
min_count = 1
size = 150
windows = 100

custom_filters = [remove_stopwords, stem_text, strip_punctuation, strip_multiple_whitespaces]
song_weight=1
title_weight=1
tag_weight=5        
tag_filename = 'model_tag_tag5.pkl'
with_w2v_model=False
w2v_model=None
save_model=True
write_results = True

  - myw2v.run
    - _get_dic(self.train, self.test, self.song_meta)

In [11]:
# 필요한 함수 정의 
def date2int(date_str):
    return int(date_str[:4]) * 10000 + int(date_str[5:7]) * 100 + int(date_str[8:10])

# 리스트를 섞어주는 함수
def shuffle_list(lst, seed=0):
    random.seed(seed)
    random.shuffle(lst)
    return lst

def tostring(lst):
    return [str(x) for x in lst]

In [12]:
# _get_dic(self, train, test, song_meta):
train = train
test = test
song_meta = song_meta

tqdm.pandas() # pandas 객체에 대한 수행 정도를 프로그래스바로 표출해주는 패키지
data = pd.concat([train, test])
data = data.set_index('id')  # 플레이리스트 id를 인덱스명으로 변환

# 플레이리스트별 곡정보(곡id)를 문자열로 변환한 후 key:플레이리스트id, value:[곡id(문자열)] 객체 생성
song_dict = data['songs'].apply(lambda x : shuffle_list(x, 123)).apply(lambda x : tostring(x)).to_dict()

# 플레이 리스트별 태그 정보를  key:플레이리스트id, value : [태그] 객체 생성
tag_dict = data['tags'].apply(lambda x : shuffle_list(x, 123)).to_dict()

# (1) tqdm.pandas()를 통해서 dataframe에서 진행 상황을 확인할 수 있게 되었으므로
# progress_apply를 사용할 수 있다.

# (2) preprocess_string()을 통해서 간단하게 문자열 전처리 가능(특수문자 제거, 단어(띄어쓰기로 구분하여 배열로 반환))

# song_dict[x['id']] ==> 플레이리스트 id가 61281인 것을 키로 하는 값(즉, 플레이리스트 수록곡)
# tag_dict[x['id']] ==> 플레이리스트 id가 61281인 것을 키로 하는 값(즉, 플레이리스트 태그들)

# preprocess_string('편하게, 잔잔하게 들을 수 있는 곡.-', custom_filters)
# ===> ['편하게', '잔잔하게', '들을', '수', '있는', '곡']

# preprocess_string('짝사랑..고백..사랑..이별..슬픔.. 감성을 자극하는곡들!', custom_filters)
# ==> ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성을', '자극하는곡들']



data = data.reset_index() # 다시 id를 컬럼으로 변환
total = data.progress_apply(lambda x : song_dict[x['id']] + tag_dict[x['id']] + preprocess_string(x['plylst_title'], custom_filters), axis = 1)

100%|██████████| 115071/115071 [00:05<00:00, 21495.77it/s]


In [71]:
print(total[92056])
data[data['id'] == 	98906]

['좋은노래', '퇴근길', '듣기좋은', '좋아서', '듣는', '노래']


,id,tags,plylst_title,songs,like_cnt,updt_date
92056,98906,"[좋은노래, 퇴근길]","듣기좋은, 좋아서 듣는 노래",[],7,2018-03-03 10:23:59.000


  - myw2v.run
    - _get_w2v(self, save_model=True):

In [14]:
# w2v_model 생성  _get_dic(train, test, song_meta)
if with_w2v_model :
            with open(w2v_model, 'rb') as f:
                w2v_model = pickle.load(f)
            print("Traning Started with " + w2v_model)
else:    
  print("Started traing Embedding... This might take about 35mins")
  time1 = time.time()

  # 워드 임베딩 학습
  w2v_model = Word2Vec(total, min_count = min_count, size = size, window = windows, sg = sg)
  time2 = time.time()
  print("Embedding completed. Took " + "{:.2f}".format((time2 - time1)/60) + "min ")
  
  if save_model:  # 생성한 w2v_model 객체 저장
      with open('./w2v_model.pkl', 'wb') as f: 
          pickle.dump(w2v_model, f)

# - _playlist2vec(self, song_weight = 1, tag_weight = 1, title_weight = 1):
# 파라미터
song_weight = song_weight
tag_weight = tag_weight
title_weight = title_weight 


p2v_model = WordEmbeddingsKeyedVectors(size) # size의 차원을 갖는 플레이리스트 별 텍스트 유사성 계산
id = []   
vec = []

 # 병합한 데이터셋의 각 로우 : 플레이리스트
for index, q in tqdm(pd.concat([train, test]).iterrows()):
    tmp_vec = 0

    # 플레이리스트 수록곡에 대한 벡터
    for song in song_dict[q['id']]:# 플레이리스트id를 키로하고 값으로 수록곡의 id를 갖는 song_dict
        try:
            tmp_vec += song_weight * w2v_model.wv.get_vector(song) / len(song_dict[q['id']])

          # print(tmp_vec) 
          # size를 30으로 지정하였기 때문에 30차원으로 벡터 생성
          # [-0.04565794 -0.00080117 -0.02168215  0.01747584  0.06025441  0.00704412
          #  -0.02204387 -0.00441014  0.01772415 -0.04369087  0.08125857 -0.00680283
          #  -0.03067778 -0.00351514  0.06477551  0.02516165  0.04594948  0.02300223
          #   0.01612392  0.03617761 -0.00340544  0.00590389  0.0270429  -0.10106849
          #  -0.08605913 -0.02068624 -0.01987263  0.02467927  0.02344197  0.03391267]
        except KeyError:
            pass

    # 플레이리스트 태그에 대한 벡터
    for tag in tag_dict[q['id']]:
        try:
            tmp_vec += tag_weight * w2v_model.wv.get_vector(tag) / len(tag_dict[q['id']])
            #  print(tmp_vec)
#             [-1.9040258   0.08954743  0.3289404   0.91529715  0.96392727 -0.2931328
#              -0.35585323  0.2304528   0.56685305 -1.4785134   0.03428814 -0.5244386
#              -0.00312252  0.45633057  1.3063939   0.20776808  1.1807116   0.5872683
#               0.6058322   0.01001734 -0.2290308  -0.18858755  0.5771195  -1.9147067
#              -1.8851228   0.20355189  0.41872978  0.2855205   0.7544385   0.29815733]
        except KeyError:
            pass
    # 플레이리스트 제목에 대한 벡터
    for title_word in preprocess_string(q['plylst_title'], custom_filters):
        try:
            tmp_vec += title_weight * w2v_model.wv.get_vector(title_word) / len(preprocess_string(q['plylst_title'], custom_filters))
        except KeyError:
            pass

    if type(tmp_vec) != int: # tmp_vec의 데이터 타입이 int가 아니면 ==> 반환된 벡터가 있다면
#         print(id)
        id.append(str(q['id'])) # 플레이리스트id에 해당하는 값을 문자열로 변환하여, id에 append
        vec.append(tmp_vec) # 각 플레이리스트의 곡, 태그, 제목을 벡터화하여 더한 값을 vec에 append
    
id = id
p2v_model.add(id, vec) # 플레이리스트 id, 플레이리스트의 속성(곡, 태그, 제목)을 임베딩하여 벡터화한 값

# _get_results(self, topn = 80, tag_filename = './w2v_tags.pkl'):
topn = topn
tag_filename = tag_filename

print("extracting results")
answers = []
tags = []
for index, q in tqdm(test.iterrows()): # test 데이터의 각 로우 : 플레이리스트
    if str(q['id']) in id:# id[]에 플레이리스트 id가 존재하는 경우,
        # p2v모델에서 test할 플레이리스트 id를 가진 것과 가장 유사한 id를 topn개 추출
        most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=topn)]
        get_song = []
        get_tag = []
        for id in most_id:
            get_song += song_dict[int(id)]# 가장 유사한 플레이리스트들의 수록곡
            get_tag += tag_dict[int(id)] # 가장 유사한 플레이리스트들의 태그들
            
        get_song = list(pd.value_counts(get_song)[:200].index) # 200곡 만 추출
        get_song = [int(x) for x in get_song] # 문자열로 작성된 곡 id를 숫자형으로 변환
        tags.append(get_tag)

        # 아래는 플레이리스트 업데이트 시간 이후 발매된 곡은 추천하지 않도록 하는 것이다.
        # 주어진 플레이리스트에 있을만한 곡을 예측하는 것이기 때문에 설정한 값인듯
        updt_date = date2int(q['updt_date'])
        song_date = song_meta.loc[get_song, 'issue_date']
        get_song = pd.Series(get_song)[[x <= updt_date for x in song_date]] 
        get_tag = list(pd.value_counts(get_tag)[:20].index)

        answers.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], get_song)[:100],
            "tags": remove_seen(q["tags"], get_tag)[:10],
        })
    else: # id[]에 플레이리스트 id가 존재하지 않는 경우,
        tags.append([])  # 빈 태그 예측
        answers.append({
          "id": most_results.loc[index]["id"],
          "songs": most_results.loc[index]['songs'],# most_popular에서 추출한 장르별 최빈도 곡
          "tags": most_results.loc[index]["tags"],# most_popular에서 추출한 최빈도 곡태그
        })
        
# check and update answer
for n, q in enumerate(answers):
  try :
    if len(q['songs'])!=100: # 추출된 예측 곡이 100개가 되지 못하면 최빈도로 구한 곡에서 몇개 가져옴
        answers[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]
    if len(q['tags'])!=10:
        answers[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]  
  except KeyError :
    pass

answers = answers
tags = tags

with open(tag_filename, 'wb') as f:
    pickle.dump(tags, f)
print("tags written to " + tag_filename)


write_results = True
if(write_results):
    write_json(answers, "./results/results.json")



Started traing Embedding... This might take about 35mins
Embedding completed. Took 41.22min 


115071it [01:29, 1288.91it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
1it [00:00,  7.29it/s]

extracting results


23015it [00:11, 1974.57it/s]


tags written to model_tag_tag5.pkl


In [73]:
# p2v_model.most_similar(str(136555), topn=topn)
p2v_model.get_vector(str(98906))


array([ 0.79248989, -4.79923201,  3.13499331,  2.2403059 , -0.71109766,
       -0.24234186,  0.81300825,  1.16993129, -0.87729043, -1.36111498,
        0.14313479,  0.6634022 ,  0.01262588, -0.86097521, -1.33925653,
       -0.69053715, -0.14812325,  0.29269749,  3.81254673, -0.99004757,
        0.54842728, -2.30717754, -2.00491118,  1.66831827, -1.56661594,
        3.29112601, -0.80598533, -2.79047704,  1.32879865,  1.42013025,
       -0.20750301, -2.3087647 , -0.5689798 ,  1.55505204, -2.35712123,
       -0.97213835,  1.98404896,  1.50481546, -0.40067652, -1.48690569,
        0.07291625, -1.60640442, -2.27486134,  0.7363804 ,  4.07068634,
        2.67006302, -1.64201844,  0.86746323, -0.73713374,  3.60533094,
       -0.84034514, -1.46857631, -1.27185941,  0.36380965, -1.39114141,
       -0.86746466, -1.08718157, -2.44982386,  0.6354633 ,  2.44016862,
       -1.52987623,  1.05234873, -3.64600825, -3.05636692,  2.08586621,
        2.61181617, -1.59590673, -0.00830192, -2.70062995,  1.31

In [62]:
test.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[좋은노래, 퇴근길]",98906,"듣기좋은, 좋아서 듣는 노래",[],7,2018-03-03 10:23:59.000


In [65]:
answer_df = pd.DataFrame(answers)
answer_df[answer_df['id'] == 136555]
answer_df.head(1)

,id,songs,tags
0,98906,"[374156, 418935, 118049, 144663, 659496, 68352...","[버스, 지하철, 퇴근, 드라이브, 기분전환, 저녁, 밤, 감성, 출근길, 휴식]"


In [66]:
real_answer_df = pd.read_json("arena_data/answers/val.json", encoding='UTF-8')
real_answer_df[real_answer_df['id'] == 136555]
real_answer_df.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[출근길, 나만알고있는노래, 운동]",98906,"듣기좋은, 좋아서 듣는 노래","[510462, 348396, 572182, 7754, 47216, 487361, ...",7,2018-03-03 10:23:59.000


ValueError: ignored

예측에 대한 성능 평가 

In [50]:
from evaluate import ArenaEvaluator
arenaEvaluator = ArenaEvaluator()
rec_fname='arena_data/results/results.json'
arenaEvaluator.evaluate(gt_fname='arena_data/answers/val.json', rec_fname=rec_fname)


Music nDCG: 0.0413319
Tag nDCG: 0.162665
Score: 0.0595319
